In [1]:
import mlflow
from dotenv import load_dotenv
import os
import sys
import pandas as pd

path=os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

# Carga las variables de entorno
load_dotenv(f"{path}/.env.mlflow_server")

# Establece la URI de MLflow
mlflow.set_tracking_uri(f"http://{os.getenv("MLFLOW_MACHINE_IP")}:8050")

model_uri = f"models:/{os.getenv("MLFLOW_MODEL_NAME")}/latest"
print(f"Cargando modelo desde {model_uri}")
model = mlflow.sklearn.load_model(model_uri)




Cargando modelo desde models:/v2_best_model/latest


d:\Python 3.12\Lib\site-packages\mlflow\store\artifact\utils\models.py:31: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
d:\Python 3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Revisión del f1 score

In [2]:
import scipy.sparse
import pandas as pd
from sklearn.metrics import f1_score


X_test = scipy.sparse.load_npz(f"{path}/data/processed/X_test.npz")
y_test = pd.read_csv(f"{path}/data/processed/y_test.csv")

# Predicción
predictions = model.predict(X_test)
print("🔮 Predicción:", predictions)


f1 = f1_score(y_test, predictions, average='weighted')
print("f1_score:", f1)



🔮 Predicción: [1 1 1 ... 1 0 1]
f1_score: 0.8413474477728583


# Predicción con datos de entrada por el usuario

In [3]:
# tomando los datos que ingresaría un usuario, por ende no tienen la transformación de datos necesaria para ser procesados por el modelo
df = pd.read_csv(f'{path}/data/user_input_example/user_input.csv', sep=';')

df.head()

,dto_igdacmlmasolicitudes,pcl_igdacmlmasolicitudes,tipo_siniestro_igdacmlmasolicitudes,fecha_siniestro_igdacmlmasolicitudes,hora_at_igatepmafurat,ind_tipo_jornada_at_igatepmafurat,ind_realizando_trabajo_hab_at_igatepmafurat,horas_previo_at_igatepmafurat,id_municipio_at_igatepmafurat,ind_sitio_ocurrencia_igatepmafurat,...,id_parte_cuerpo_igatepmafurat,id_agente_at_igatepmafurat,id_mecanismo_at_igatepmafurat,ind_testigo_at_igatepmafurat,descripcion_at_igatepmafurat,id_medio_recepcion_igatepmafurat,centro_trabajo_igual_igatepmafurat,accidente_grave_igatepmafurat,riesgo_biologico_igatepmafurat,id_sitio_ocurrencia_igatepmafurat
0,s,s,0,2009-03-05 00:00:00+00:00,9,1,s,2,1,2,...,446,336,9,1,el trabajador se encontraba realizando su labo...,5,1,n,n,1
1,s,s,0,2009-03-04 00:00:00+00:00,1,1,s,7,1,2,...,3,5,3,0,el trabajador se encontraba lavando el chasis ...,5,1,n,n,1
2,s,s,0,2009-03-05 00:00:00+00:00,13,1,s,5,1,2,...,4,5,1,0,el trabajador se encontraba sentandose en una ...,5,1,n,n,1
3,s,s,0,2009-03-04 00:00:00+00:00,11,1,s,6,1,2,...,112,4,9,0,el trabajador se encontraba en la labor habitu...,5,1,n,n,1
4,s,s,0,2009-03-05 00:00:00+00:00,13,1,s,5,1,7,...,4,5,1,1,el trabajador se encontraba en sus labores hab...,5,1,n,n,1


In [ ]:
from transform_input import prepare_input_data

# vamos a tomar solo dos filas para hacer pruebas
df_sample = df.iloc[:2].copy()

data=prepare_input_data(df_sample).copy()


2025-03-11 13:49:07,339 - INFO - 	Se han remapeado los valores de la columna 'ind_realizando_trabajo_hab_at_igatepmafurat'.
2025-03-11 13:49:07,345 - INFO - 	Se han ajustado las columnas de categoría.


2025-03-11 13:49:07,350 - INFO - 	Se han imputado los espacios vacíos o ceros en variables binarias.
2025-03-11 13:49:07,356 - INFO - 	Se han extraído las variables temporales 'mes' y 'día' del siniestro.
2025-03-11 13:49:07,365 - INFO - 	Se han capturado las periodicidades en las variables temporales con seno y coseno.


🔍 Cargando el pipeline desde: d:\OneDrive - Universidad de La Salle\Maestría IA\S4\Desarrollo de soluciones\Proyecto\Repo desarrollo soluciones\src\data_preprocessing\trained_pipelines\transformation_pipeline.pkl
✅ Pipeline cargado exitosamente.


d:\Python 3.12\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [5]:
data.shape

(2, 303)

In [6]:

predictions = model.predict(data)
print("🔮 Predicción:", predictions)


🔮 Predicción: [0 1]
